In [ ]:
%%capture
!pip install yt-dlp youtube_search_python gspread pydrive2 google-auth #google-auth-oauthlib google-auth-httplib2

# PARAMETERS

In [ ]:
QUERY_SHEET_ID = "1ZpVOor2FWBvX-bUp7NTX4CyoTRxJj2NclLSlYIRgdtc"
VIDEO_SHEET_ID = "1BBmV1Asb2brp7mzvukwYcG49rZT99iS0t46OCqJDSak"


VIDEO_OUTPUT_PATH = "/content/drive/MyDrive/FLAWLESS_ROOT_DIRECTORY/Data/raw_videos/"

# SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Google Sheets - Video Sheet and Search Queries Sheet

In [ ]:
from google.colab import auth
from google.auth import default
import google.auth.transport.requests
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import gspread
import pandas as pd

class QuerySheet:

  def __init__(self, FILE_ID):
    auth.authenticate_user()
    self.creds, _ = default(scopes=['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets'])
    gauth = GoogleAuth()
    gauth.credentials = self.creds
    drive = GoogleDrive(gauth)
    self.gc = gspread.authorize(self.creds)

    spreadsheet = self.gc.open_by_key(FILE_ID)
    self.worksheet = spreadsheet.sheet1

    data = self.worksheet.get_all_values()
    headers = data.pop(0)
    self.df = pd.DataFrame(data, columns=headers)

  def get_df(self):
    return self.df

  def mark_query_processed(self, text):
    self.df.loc[self.df['QUERY'] == text, 'PROCESSED?'] = 1

  def get_new_queries(self):
    return self.df[self.df.iloc[:, 1] != "1"].iloc[:, 0].tolist()

  def update_sheet(self):
    self.worksheet.clear()
    data = [self.df.columns.values.tolist()] + self.df.values.tolist()
    self.worksheet.update('A1', data)


class VideoSheet:

  def __init__(self, VIDEO_CSV_ID):
    auth.authenticate_user()
    self.creds, _ = default(scopes=['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets'])
    gauth = GoogleAuth()
    gauth.credentials = self.creds
    drive = GoogleDrive(gauth)
    self.gc = gspread.authorize(self.creds)

    spreadsheet = self.gc.open_by_key(VIDEO_CSV_ID)
    self.worksheet = spreadsheet.sheet1

    data = self.worksheet.get_all_values()
    headers = data.pop(0)
    self.df = pd.DataFrame(data, columns=headers)

  def add_video_data(self, df):
    next_row = len(self.worksheet.get_all_values()) + 1
    # Convert DataFrame to a list of lists, include header if sheet is completely empty
    if next_row == 1:
        data = [df.columns.values.tolist()] + df.values.tolist()
    else:
        data = df.values.tolist()
    # Update worksheet starting from the next empty row
    self.worksheet.update(f'A{next_row}', data)

  def update_sheet(self):
    self.worksheet.clear()
    data = [self.df.columns.values.tolist()] + self.df.values.tolist()
    self.worksheet.update('A1', data)


## Youtube Functions

In [ ]:
import pandas as pd
from youtubesearchpython import VideosSearch
import json

def get_youtube_data(query, results_limit=5):
    video_search = VideosSearch(query, limit=results_limit)
    results = video_search.result()
    return [x['id'] for x in results['result'] ]

    # returns a bunch of ID's

import yt_dlp
import pandas as pd

def get_video_info(url):

    ydl_opts = {
        'skip_download': True,  # Do not download the video
        'force_generic_extractor': True,  # This might be necessary if yt-dlp is unable to download video info directly
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        # Fetch video information
        info_dict = ydl.extract_info(url, download=False)
        # Since '--print-json' is not directly available, we work with the info_dict returned
        print(info_dict)
        return info_dict

def produce_video_record(url):

    # try except to account for video being copyrighted
    try:
        info = get_video_info(url)
        record = pd.Series([info.get('id'), info.get('title', 'N/A'), url, f"{info.get('width', 'N/A')}x{info.get('height', 'N/A')}", info.get('duration', 'N/A'), info.get('language', 'N/A')],
            index = ['VIDEO ID', 'Name', 'Video Link', 'Resolution', 'Duration', 'Language'])
        return record
    except Exception as e:
        print("Could not get video url: ", e)
        return pd.Series(['N/A', 'N/A', url, 'N/A', 'N/A', 'N/A'], index=['VIDEO ID', 'Name', 'Video Link', 'Resolution', 'Duration', 'Language'])



def download_video(video_url, desired_name, output_path):
    ydl_opts = {
        'format': 'best[ext=mp4]',
        'outtmpl': VIDEO_OUTPUT_PATH + desired_name + '.%(ext)s',
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([video_url])
            print("Video downloaded successfully.")
        except Exception as e:
            print("An error occurred while downloading the video: ", e)

# Execution

In [ ]:
# testing new loop logic
VS = VideoSheet(VIDEO_SHEET_ID)
QS = QuerySheet(QUERY_SHEET_ID)

VIDEO_ID_QUEUE = []

search_queries = QS.get_new_queries()

for query in search_queries:
    VIDEO_ID_QUEUE += get_youtube_data(query, results_limit = 4)




In [ ]:
len(search_queries)

3438

In [ ]:
search_queries

['Godzilla clips',
 'Godzilla scene',
 'Godzilla opening',
 'Godzilla ending',
 'Godzilla trailer',
 '哥斯拉 clips',
 '哥斯拉 scene',
 '哥斯拉 opening',
 '哥斯拉 ending',
 '哥斯拉 trailer',
 'Годзилла clips',
 'Годзилла scene',
 'Годзилла opening',
 'Годзилла ending',
 'Годзилла trailer',
 'Γκοτζίλα clips',
 'Γκοτζίλα scene',
 'Γκοτζίλα opening',
 'Γκοτζίλα ending',
 'Γκοτζίλα trailer',
 'גודזילה clips',
 'גודזילה scene',
 'גודזילה opening',
 'גודזילה ending',
 'גודזילה trailer',
 'ก็อดซิลล่า clips',
 'ก็อดซิลล่า scene',
 'ก็อดซิลล่า opening',
 'ก็อดซิลล่า ending',
 'ก็อดซิลล่า trailer',
 'Godzila clips',
 'Godzila scene',
 'Godzila opening',
 'Godzila ending',
 'Godzila trailer',
 'Годзила clips',
 'Годзила scene',
 'Годзила opening',
 'Годзила ending',
 'Годзила trailer',
 'GODZILLA ゴジラ clips',
 'GODZILLA ゴジラ scene',
 'GODZILLA ゴジラ opening',
 'GODZILLA ゴジラ ending',
 'GODZILLA ゴジラ trailer',
 '고질라 clips',
 '고질라 scene',
 '고질라 opening',
 '고질라 ending',
 '고질라 trailer',
 'Ґодзілла clips',
 'Ґодзілла scene

In [ ]:
len(VIDEO_ID_QUEUE)

13664

In [ ]:
VIDEO_ID_QUEUE

['AkcfB3z0_-0',
 'KIKEHZCL5Nc',
 '5BxqahE0fu8',
 'QsvRms3HJi4',
 'AkcfB3z0_-0',
 '5BxqahE0fu8',
 '3MNWGN905Co',
 'KIKEHZCL5Nc',
 'NBQJjqnG1iI',
 '1b2CtgkwJQA',
 'RUHHxecmLUI',
 'xFLlrcLbxHU',
 'BWjxfzFnGRQ',
 'JXTdArIvggI',
 'bs1Wkl7m1qs',
 'EycQn0z61VY',
 'vIu85WQTPRc',
 'qqrpMRDuPfc',
 'wVDtmouV9kM',
 'QFxN2oDKk0E',
 'zkpMEzz19RA',
 'K1htLxKV3bs',
 'RobDdJNS-Gg',
 'kfe_pvnFFE0',
 'AkcfB3z0_-0',
 'zkpMEzz19RA',
 'abjCORW2SIo',
 'ZYwDPnPKLIs',
 'RobDdJNS-Gg',
 'GomKKHPKJFA',
 'cD0FeOrK6rk',
 'msdFDqV4oi4',
 '2qquWqyB5LQ',
 'IIdmPW3Bozs',
 'm6ng-7OxvNg',
 'CqwgFgPRQw8',
 'zkpMEzz19RA',
 'K1htLxKV3bs',
 's1DUC_wBUvY',
 'rG74w_-abrA',
 'AkcfB3z0_-0',
 'r_0JjYUe5jo',
 '42poB7JbV00',
 'nf7GsKFepDg',
 'AkcfB3z0_-0',
 '42poB7JbV00',
 '5BxqahE0fu8',
 'XQRDcf6f6Ls',
 'NBQJjqnG1iI',
 '1b2CtgkwJQA',
 'mBlPuHK7W-4',
 'RUHHxecmLUI',
 '42poB7JbV00',
 'BWjxfzFnGRQ',
 'EycQn0z61VY',
 'bs1Wkl7m1qs',
 'vIu85WQTPRc',
 'qqrpMRDuPfc',
 'pt6bMHUoE6I',
 'lV1OOlGwExM',
 'B0UZs9Gj_Ko',
 'odM92ap8_c0',
 'U2pYos

In [ ]:
print(len(search_queries) * 4)
print(len(VIDEO_ID_QUEUE))

13752
17076


In [ ]:
RESULTS_LIMIT = 5

VS = VideoSheet(VIDEO_SHEET_ID)
QS = QuerySheet(QUERY_SHEET_ID)

VIDEO_ID_QUEUE = []

search_queries = QS.get_new_queries()

for query in search_queries:
    VIDEO_ID_QUEUE += get_youtube_data(query, results_limit = 5)

NEW_VIDEOS_DF = pd.DataFrame(columns = ['VIDEO ID', 'Name', 'Video Link', 'Resolution', 'Duration', 'Language'])

In [ ]:
%%capture

for i in range(len(VIDEO_ID_QUEUE)):
    id = VIDEO_ID_QUEUE[i]
    url = "https://www.youtube.com/watch?v=" + id

    download_video(url, id, VIDEO_OUTPUT_PATH)

    record = produce_video_record(url)
    NEW_VIDEOS_DF.loc[-1] = record  # adding a row
    NEW_VIDEOS_DF.index = NEW_VIDEOS_DF.index + 1  # shifting index
    NEW_VIDEOS_DF = NEW_VIDEOS_DF.sort_index()  # sorting by index

    # mark query sheet for every 5 videos

    if (i%5 == 4):
      query = search_queries[int(i/5)]
      QS.mark_query_processed(query)
      QS.update_sheet()

    if (i%11 == 10):
      VS.add_video_data(NEW_VIDEOS_DF)


VS.add_video_data(NEW_VIDEOS_DF)

# for id in VIDEO_ID_QUEUE:

#     url = "https://www.youtube.com/watch?v=" + id

#     download_video(url, id, VIDEO_OUTPUT_PATH)
#     # mark on query sheet here

#     record = produce_video_record(url)
#     NEW_VIDEOS_DF.loc[-1] = record  # adding a row
#     NEW_VIDEOS_DF.index = NEW_VIDEOS_DF.index + 1  # shifting index
#     NEW_VIDEOS_DF = NEW_VIDEOS_DF.sort_index()  # sorting by index

# for query in search_queries:
#     QS.mark_query_processed(query)

# VS.add_video_data(NEW_VIDEOS_DF)
# QS.update_sheet()

APIError: {'code': 400, 'message': 'Range (Sheet1!A20333) exceeds grid limits. Max rows: 20332, max columns: 29', 'status': 'INVALID_ARGUMENT'}

In [ ]:
while True:pass

In [ ]:
int(6/5)

1